In [31]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from brokenaxes import brokenaxes

import itertools

In [2]:
with open('eleuther_eval_jobs.tsv', 'r') as f:
    df1 = pd.read_csv(f, sep='\t', index_col=None)

df1['eval_method'] = 'eleuther'
with open('composer_eval_jobs.tsv', 'r') as f:
    df2 = pd.read_csv(f, sep='\t', index_col=None)

df2['eval_method'] = 'composer'

df = pd.concat([df1, df2])
df

,gpu_num,parameters-icl_tasks,parameters-model,run_name,run_time,accuracy,eval_method
0,64,piqa,125m,gpt-eval-gpu-num-64-icl-tasks-piqa-model-125m-...,718.886351,0.628945,eleuther
1,64,piqa,1.3b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther
2,64,piqa,2.7b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-2-7b-...,3260.289173,0.731774,eleuther
3,32,piqa,125m,gpt-eval-gpu-num-32-icl-tasks-piqa-model-125m-...,718.886351,0.628945,eleuther
4,32,piqa,1.3b,gpt-eval-gpu-num-32-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther
...,...,...,...,...,...,...,...
46,8,piqa,1.3b,gpt-eval-gpu-num-8-icl-tasks-piqa-model-1-3b-1554,49.531831,0.725543,composer
47,8,piqa,2.7b,gpt-eval-gpu-num-8-icl-tasks-piqa-model-2-7b-1155,84.149821,0.737500,composer
48,1,piqa,125m,gpt-eval-gpu-num-1-icl-tasks-piqa-model-125m-7841,100.978092,0.623504,composer
49,1,piqa,1.3b,gpt-eval-gpu-num-1-icl-tasks-piqa-model-1-3b-2736,390.384548,0.704570,composer


In [3]:
df = df.dropna()
graph_partition_cols = ['parameters-icl_tasks', 'parameters-model'] # diff values will be on diff graphs
line_partition_cols = ['eval_method'] # diff values will be diff lines

graph_partition_vals = {
    k: set(df[k]) for k in graph_partition_cols
}
line_partition_vals = {
    k: set(df[k]) for k in line_partition_cols
}
x_col = 'gpu_num'
y_col = 'run_time'
df.sort_values(by=['gpu_num'], inplace=True)
df

/var/folders/v7/1wjc57v9783frf5llnl1vgph0000gn/T/ipykernel_23792/1736634824.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=['gpu_num'], inplace=True)


,gpu_num,parameters-icl_tasks,parameters-model,run_name,run_time,accuracy,eval_method
14,1,lambada,2.7b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-2-7...,931.951627,0.622162,composer
48,1,piqa,125m,gpt-eval-gpu-num-1-icl-tasks-piqa-model-125m-7841,100.978092,0.623504,composer
49,1,piqa,1.3b,gpt-eval-gpu-num-1-icl-tasks-piqa-model-1-3b-2736,390.384548,0.704570,composer
13,1,lambada,1.3b,gpt-eval-gpu-num-8-icl-tasks-lambada-model-1-3...,546.160601,0.571900,composer
12,1,lambada,125m,gpt-eval-gpu-num-8-icl-tasks-lambada-model-125...,140.854822,0.375121,composer
...,...,...,...,...,...,...,...
16,128,lambada,1.3b,gpt-eval-gpu-num-128-icl-tasks-lambada-model-1...,5.198331,0.570694,composer
17,128,lambada,2.7b,gpt-eval-gpu-num-128-icl-tasks-lambada-model-2...,8.393714,0.621570,composer
18,256,lambada,125m,gpt-eval-gpu-num-256-icl-tasks-lambada-model-1...,1.380043,0.375744,composer
19,256,lambada,1.3b,gpt-eval-gpu-num-256-icl-tasks-lambada-model-1...,3.137048,0.572173,composer


In [4]:
def partition_graphs(df, graph_partition_vals):
    keys, vals = graph_partition_vals.keys(), graph_partition_vals.values()
    
    dataset_graph_partitions = []
    for combo in itertools.product(*vals):
        filter_string = ""
        
        df_part = {}
        df_part['choices'] = {}
        
        for col_name, col_val in zip(keys, combo):
            filter_string += f"`{col_name}` == '{col_val}' and"
            df_part['choices'][col_name] = col_val
            
        filter_string = filter_string[:-4]
        df_subset = df.query(filter_string)
        df_part['df'] = df_subset
        dataset_graph_partitions.append(df_part)
    
    return dataset_graph_partitions



graph_partitions = partition_graphs(df, graph_partition_vals)

In [5]:
def partition_lines(df, line_partition_vals):
    keys, vals = line_partition_vals.keys(), line_partition_vals.values()
    dataset_line_partitions = []
    for combo in itertools.product(*vals):
        filter_string = ""
        
        df_part = {}
        df_part['choices'] = {}
        
        for col_name, col_val in zip(keys, combo):
            filter_string += f"`{col_name}` == '{col_val}' and"
            df_part['choices'][col_name] = col_val
            
        filter_string = filter_string[:-4]
        df_subset = df.query(filter_string)
        df_part['df'] = df_subset
        dataset_line_partitions.append(df_part)
    return dataset_line_partitions 

def make_title(choices):
    if 'eval_method' in choices:
        return f"{choices['eval_method']}-based eval, {choices['parameters-icl_tasks'].upper()}"
    elif 'parameters-model' in choices:
        return f"{choices['parameters-model']} parameter model, {choices['parameters-icl_tasks'].upper()}"
    else:
        return None
   



In [59]:


for idx, graph_info in enumerate(graph_partitions):
    choices = graph_info['choices']
    line_partitions = partition_lines(graph_info['df'], line_partition_vals)
    legend = []
    
    xmin, xmax = (0, 256)
    ylims = [[], []]
    
    for line in line_partitions:
        if line['choices']['eval_method'] == 'composer':
            ylims[0] = (0, int(max(line['df'][y_col])+50))
            
        else:
            ylims[1] = (int(max(line['df'][y_col])-200), int(max(line['df'][y_col])+50))

    print(ylims)
    bax = brokenaxes(xlims=[(xmin, xmax),], ylims=tuple(ylims))
    
    for line in line_partitions:
        legend.append(', '.join(line['choices'].values()))
        print(line['choices'])
        
        if line['choices']['eval_method'] == 'composer':
            bax.plot(line['df'][x_col], line['df'][y_col])
        else:
            bax.hlines(max(line['df'][y_col]), xmin=xmin, xmax=xmax, color='orange', linestyle='--')


        # hide the spines between ax and ax2
       
    
    
    plt.legend(legend)
   
    
    plt.title(make_title(choices))
    plt.xlabel('Number of GPUs')
    plt.ylabel('Run time (seconds)')
    plt.savefig(f"fig{idx}.png", )
    plt.clf()
    break



[(0, 981), (4331, 4581)]
{'eval_method': 'composer'}
{'eval_method': 'eleuther'}


<Figure size 640x480 with 0 Axes>

In [7]:
line['df']

,gpu_num,parameters-icl_tasks,parameters-model,run_name,run_time,accuracy,eval_method
10,8,piqa,1.3b,gpt-eval-gpu-num-8-icl-tasks-piqa-model-1-3b-1503,1927.111743,0.707835,eleuther
7,16,piqa,1.3b,gpt-eval-gpu-num-16-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther
4,32,piqa,1.3b,gpt-eval-gpu-num-32-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther
1,64,piqa,1.3b,gpt-eval-gpu-num-64-icl-tasks-piqa-model-1-3b-...,1927.111743,0.707835,eleuther


In [8]:
x,y = zip(*big_models)

NameError: name 'big_models' is not defined

In [ ]:
big_models = [(7e10,110.333),
(2.02e11,340.168),
(3.23e11,544.74),
(4.43e11,741.209)]


def billions_formatter(x, pos):
    return f'{x / 1000000000}'

In [ ]:

fig, ax = plt.subplots()
ax.plot(x,y)
ax.xaxis.set_major_formatter(FuncFormatter(billions_formatter))
plt.title("LAMBADA eval time on 64 GPUs")
plt.xlabel('Number of Parameters (billions)')
plt.ylabel('Run time (seconds)')
plt.savefig(f"fig4.png", )
plt.clf()